# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Parameters" data-toc-modified-id="Parameters-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Parameters</a></div><div class="lev1 toc-item"><a href="#Creating-template" data-toc-modified-id="Creating-template-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating template</a></div><div class="lev2 toc-item"><a href="#Hide-Inputs" data-toc-modified-id="Hide-Inputs-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Hide Inputs</a></div><div class="lev2 toc-item"><a href="#Theme" data-toc-modified-id="Theme-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Theme</a></div><div class="lev2 toc-item"><a href="#Logo" data-toc-modified-id="Logo-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Logo</a></div><div class="lev2 toc-item"><a href="#Concatenate-Template" data-toc-modified-id="Concatenate-Template-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Concatenate Template</a></div><div class="lev2 toc-item"><a href="#Write-File" data-toc-modified-id="Write-File-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Write File</a></div><div class="lev1 toc-item"><a href="#Run-nbconvert" data-toc-modified-id="Run-nbconvert-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Run nbconvert</a></div>

This NoteBook aims at explaining the process for converting a Rise-ipynb into HTML. It can also be used to make the conversion by setting up the *parameters* cell, then running all cells.

# Parameters

Evaluate the cell below to define your parameters.

In [15]:
# Location of the notebook
nbprefix = "NoteBookPresentation"
# Relies on a local copy of reveal.js (otherwise use a CDN)
localreveal = False
# What to do with the inputs hidden by the nbextension hide inputs (keep/remove/toggle)
hidden_inputs = "toggle"
# Set up a logo (lincs/nokia/none)
logo = "lincs"
# Set up theme
theme = "sky"
# Embed images in base64; currently require inlinehtmlimages-v1.7.exe
embed = True
# Other options, see https://github.com/hakimel/reveal.js#configuration
revealoptions = """
    controls: true,
    progress: true,
    history: false,
    width: "100%",
    height: "100%",
    margin: 0.1,
    minScale: .1,
    maxScale: 10,

    theme: Reveal.getQueryHash().theme, // available themes are in /css/theme
    transition: 'page', //Reveal.getQueryHash().transition || 'linear', // default/cube/page/concave/zoom/linear/none
"""

# Creating template

We create a template **myplate.tpl** in current directory that adapts the reveal template from **\Lib\site-packages\nbconvert\templates\html\slides_reveal.tpl** so it embeds all the things we need.

## Hide Inputs

- If hidden_inputs is set to "remove", all hidden inputs will be removed.
- If set to "toggle", all inputs are preserved and can be hidden/revealed by clicking the corresponding output. Hidden inputs will be initially... hidden!

In [16]:
hidden_str = ""
hidden_script = ""
if hidden_inputs == "toggle":
    hidden_str = """
{%- block input_group -%}
{%- if cell.metadata.hide_input -%}
<div class="hideable_input" style="display:none">
{{ super() }}
</div>
{%- else -%}
<div class="hideable_input">
{{ super() }}
</div>
{%- endif -%}
{%- endblock input_group -%}
"""
    hidden_script = """
<script>
$(document).ready(function(){
  $(".output_wrapper").click(function(){
      $(this).prev('.hideable_input').slideToggle();
  });
})
</script>
"""
elif hidden_inputs == "remove":
    hidden_str = """
{%- block input_group -%}
{%- if cell.metadata.hide_input -%}
{%- else -%}
{{ super() }}
{%- endif -%}
{%- endblock input_group -%}
"""


## Theme

In [17]:
theme_str = """<link rel="stylesheet" href="{{resources.reveal.url_prefix}}/css/theme/"""+theme+""".css" id="theme">"""

## Logo

In [18]:
logo_str = """<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
"""
if logo == "lincs":
    logo_str += """
body:after {
background-image: url('lincs.png');
background-size: 200px 100px;
position: fixed;
bottom: 1em;
right: 8em;
width: 200px; 
height: 100px;
content:"";
}
"""
elif logo == "nokia":
    logo_str = """
body:after {
background-image: url('nokia.png');
background-size: 200px 60px;
position: fixed;
bottom: 2.8em;
right: 8em;
width: 200px; 
height: 60px;
content:"";
}
"""

logo_str += """</style>""" 

## Concatenate Template

In [19]:
template = """
{%- extends 'basic.tpl' -%}
{% from 'mathjax.tpl' import mathjax %}

{%- block any_cell scoped -%}
{%- if cell.metadata.get('slide_start', False) -%}
<section>
{%- endif -%}
{%- if cell.metadata.get('subslide_start', False) -%}
<section>
{%- endif -%}
{%- if cell.metadata.get('fragment_start', False) -%}
<div class="fragment">
{%- endif -%}

{%- if cell.metadata.slide_type == 'notes' -%}
<aside class="notes">
{{ super() }}
</aside>
{%- elif cell.metadata.slide_type == 'skip' -%}
{%- else -%}
{{ super() }}
{%- endif -%}

{%- if cell.metadata.get('fragment_end', False) -%}
</div>
{%- endif -%}
{%- if cell.metadata.get('subslide_end', False) -%}
</section>
{%- endif -%}
{%- if cell.metadata.get('slide_end', False) -%}
</section>
{%- endif -%}

{%- endblock any_cell -%}
"""
template += hidden_str
template += """
{% block header %}
<!DOCTYPE html>
<html>
<head>

<meta charset="utf-8" />
<meta http-equiv="X-UA-Compatible" content="chrome=1" />

<meta name="apple-mobile-web-app-capable" content="yes" />
<meta name="apple-mobile-web-app-status-bar-style" content="black-translucent" />

<title>{{resources['metadata']['name']}} slides</title>

<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.1.10/require.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/2.0.3/jquery.min.js"></script>

<!-- General and theme style sheets -->
<link rel="stylesheet" href="{{resources.reveal.url_prefix}}/css/reveal.css">
"""
template += theme_str
template += """
<!-- If the query includes 'print-pdf', include the PDF print sheet -->
<script>
if( window.location.search.match( /print-pdf/gi ) ) {
        var link = document.createElement( 'link' );
        link.rel = 'stylesheet';
        link.type = 'text/css';
        link.href = '{{resources.reveal.url_prefix}}/css/print/pdf.css';
        document.getElementsByTagName( 'head' )[0].appendChild( link );
}

</script>

<!--[if lt IE 9]>
<script src="{{resources.reveal.url_prefix}}/lib/js/html5shiv.js"></script>
<![endif]-->

<!-- Loading the mathjax macro -->
{{ mathjax() }}

<!-- Get Font-awesome from cdn -->
<link rel="stylesheet" href="//netdna.bootstrapcdn.com/font-awesome/4.1.0/css/font-awesome.css">

{% for css in resources.inlining.css -%}
    <style type="text/css">
    {{ css }}
    </style>
{% endfor %}
"""
template += logo_str
template += """
<style type="text/css">
/* Overrides of notebook CSS for static HTML export */
.reveal {
  font-size: 160%;
  overflow-y: hidden;
}
.reveal pre {
  width: inherit;
  padding: 0.4em;
  margin: 0px;
  font-family: monospace, sans-serif;
  font-size: 80%;
  box-shadow: 0px 0px 0px rgba(0, 0, 0, 0);
}
.reveal pre code {
  padding: 0px;
}
.reveal section img {
  border: 0px solid black;
  box-shadow: 0 0 10px rgba(0, 0, 0, 0);
}
.reveal i {
  font-style: normal;
  font-family: FontAwesome;
  font-size: 2em;
}
.reveal .slides {
  text-align: left;
}
.reveal.fade {
  opacity: 1;
}
.reveal .progress {
  position: static;
}
div.input_area {
  padding: 0.06em;
}
div.code_cell {
  background-color: transparent;
}
div.prompt {
  width: 11ex;
  padding: 0.4em;
  margin: 0px;
  font-family: monospace, sans-serif;
  font-size: 80%;
  text-align: right;
}
div.output_area pre {
  font-family: monospace, sans-serif;
  font-size: 80%;
}
div.output_prompt {
  /* 5px right shift to account for margin in parent container */
  margin: 5px 5px 0 0;
}
div.text_cell.rendered .rendered_html {
  /* The H1 height seems miscalculated, we are just hidding the scrollbar */
  overflow-y: hidden;
}
a.anchor-link {
  /* There is still an anchor, we are only hidding it */
  display: none;
}
.rendered_html p {
  text-align: inherit;
}
</style>
"""
template += hidden_script
template += """
</head>
{% endblock header%}


{% block body %}
{% block pre_slides %}
<body>
{% endblock pre_slides %}

<div class="reveal">
<div class="slides">
{{ super() }}
</div>
</div>
{% block post_slides %}
<script>

require(
    {
      // it makes sense to wait a little bit when you are loading
      // reveal from a cdn in a slow connection environment
      waitSeconds: 15
    },
    [
      "{{resources.reveal.url_prefix}}/lib/js/head.min.js",
      "{{resources.reveal.url_prefix}}/js/reveal.js"
    ],

    function(head, Reveal){

        // Full list of configuration options available here: https://github.com/hakimel/reveal.js#configuration
        Reveal.initialize({
"""
template += revealoptions
template += """

            // Optional libraries used to extend on reveal.js
            dependencies: [
                { src: "{{resources.reveal.url_prefix}}/lib/js/classList.js",
                  condition: function() { return !document.body.classList; } },
                { src: "{{resources.reveal.url_prefix}}/plugin/notes/notes.js",
                  async: true,
                  condition: function() { return !!document.body.classList; } }
            ]
        });

        var update = function(event){
          if(MathJax.Hub.getAllJax(Reveal.getCurrentSlide())){
            MathJax.Hub.Rerender(Reveal.getCurrentSlide());
          }
        };

        Reveal.addEventListener('slidechanged', update);

        var update_scroll = function(event){
          $(".reveal").scrollTop(0);
        };

        Reveal.addEventListener('slidechanged', update_scroll);

    }
);
</script>

</body>
{% endblock post_slides %}
{% endblock body %}

{% block footer %}
</html>
{% endblock footer %}
"""

## Write File

We write a template having the same name than the file we want to convert.

In [20]:
with open(nbprefix+".tpl", "w") as f:
    f.write(template)

# Run nbconvert

To embed the images, the current workflow uses a Windows exe. In a further version, the embedding should be made in Python so it will be platform independent.

In [21]:
import subprocess as sub

if localreveal:
    reveal = "reveal.js"
else:
    reveal = "https://cdn.bootcss.com/reveal.js/3.3.0"

import os
sub.run("jupyter nbconvert --to slides "+nbprefix+".ipynb --template "+nbprefix+" --reveal-prefix "+reveal)
if embed:
    try:
        os.remove(nbprefix+".standalone.html")
    except:
        print("No Prezi")
    sub.run("inlinehtmlimages-v1.7.exe -in "+nbprefix+".slides.html -out "+nbprefix+".standalone.html")